In [1]:
from numpy import *; from numpy.core.records import fromarrays
from matplotlib.pyplot import *
import sys; sys.path.insert(0,'/home/noah/Desktop/large_analysis/ca_analysis/'); import os
import predefined as pd; import fileread as fr
import pickle
import time

with open('/home/noah/Desktop/large_analysis/ca_analysis/detector_response/detector_response.pickle','rb') as f:
    detector_response=pickle.load(f)           

In [2]:
nbins,rang=2000,[0,800]

path='/home/noah/Desktop/large_analysis/zeck_simulation/ca_small_set/'
b0,b1=load(path+'45Ca_b0.npy'),load(path+'45Ca_b1.npy')               # Reading in 45Ca b=1 b=0 PENELOPE spectra
b0,b1=pd.sim_restructure(b0),pd.sim_restructure(b1)                   # Restructuring the sim data
b0,b1=b0.view(recarray),b1.view(recarray)                             # Making the spectra record arrays (easier)
Fullb0,Fullb1=pd.sim_comb_single_pixel(b0),pd.sim_comb_single_pixel(b1)  #Perfect summed simulated spectra
Fullb0,Fullb1=Fullb0.view(recarray),Fullb1.view(recarray)
Fullb0_hist=histogram(Fullb0.energy,bins=nbins,range=rang)[0]
Fullb1_hist=histogram(Fullb1.energy,bins=nbins,range=rang)[0]
Norm_b0 = 1./np.sum(Fullb0_hist)        # This is used later when taking difference of b=1 and b=0 spectra
Norm_b1 = 1./np.sum(Fullb1_hist)        # This is used later when taking difference of b=1 and b=0 spectra

#### Making simulated single pixel spectra
Note: the smearing is done here. E-->E+deltaE, where deltaE=np.random.normal(loc=0,scale=sigma)

In [3]:
thresh=50                     # in keV
coinc_window=400e-9
bdchs=[11,12,35,36]                            #This includes 52,64,65W and 64E
# bdchs=[11]                            #This includes 52,64,65W and 64E

truth0,truth1=np.zeros(len(b0),dtype=bool),np.zeros(len(b1),dtype=bool)
for bdch in bdchs:                     #Taking sim events ONLY for bdchs
    bd,ch=int(bdch/8),int(bdch%8)
    truth0+=(b0.board==bd)*(b0.channel==ch)
    truth1+=(b1.board==bd)*(b1.channel==ch)
sb0,sb1=b0[truth0],b1[truth1]
sb0,sb1=sb0.view(np.recarray),sb1.view(np.recarray)

for bdch in bdchs:
    bd,ch=int(bdch/8),int(bdch%8)
    sigma=detector_response[bdch]                 # Detector response for bd,ch
    trutharray=(sb0.board==bd)*(sb0.channel==ch)  # events in sb0= with bd ch
    sb0.energy[trutharray] += np.random.normal(loc=0,scale=sigma,size=np.sum(trutharray))
    trutharray=(sb1.board==bd)*(sb1.channel==ch)  # events in sb0= with bd ch
    sb1.energy[trutharray] += np.random.normal(loc=0,scale=sigma,size=np.sum(trutharray))
    
sb0=sb0[sb0.energy>thresh]                          #Applying threshold cut to sim
sb1=sb1[sb1.energy>thresh]                    #Applying threshold cut to sim

sb0=pd.sim_sing_pix_coinc(sb0,coinc_window) #Summing events within a coinc_window timewindow
sb0=sb0[pd.sim_single_event(sb0)]           # Making single pixel sim spectra
sb1=pd.sim_sing_pix_coinc(sb1,coinc_window) #Summing events within a coinc_window timewindow
sb1=sb1[pd.sim_single_event(sb1)]           # Making single pixel sim spectra

#### Making simulated single pixel histograms with smearing

In [4]:
# nbins,rang= 1000, (0,800)                           # 0.8 keV wide bins
b0_hist,bins=histogram([],bins=nbins,range=rang)   #b0 single pixel histogram
b1_hist,bins=histogram([],bins=nbins,range=rang)   #b1 single pixel histogram
b0_hist,b1_hist=b0_hist.astype(float),b1_hist.astype(float)
bins=pd.cbins(bins)

# bdchs_of_interest=bdchs

bdchs_of_interest=[35]
# bdchs_of_interest=[11]
# bdchs_of_interest=[12]
# bdchs_of_interest=[35,12]
# bdchs_of_interest=[11,12,35]

for bdch in bdchs_of_interest:             # Looping over bdchs to smear sim pixel-by-pixel
    bd,ch=int(bdch/8),int(bdch%8)
    trutharray=(sb0.board==bd)*(sb0.channel==ch)  # events in sb0= with bd ch
    b0_hist+=histogram(sb0.energy[trutharray],bins=nbins,range=rang)[0]
    
    trutharray=(sb1.board==bd)*(sb1.channel==ch)  # events in sb1= with bd ch
    b1_hist+=histogram(sb1.energy[trutharray],bins=nbins,range=rang)[0]
b0_hist*=Norm_b0
b1_hist*=Norm_b1                            # Normalizing the total number of counts in b1 to that of b1

In [14]:
bins=np.arange(rang[1]/2./nbins,rang[1],rang[1]/nbins).astype('f4')

fs=30
bdch=11
bd,ch=divmod(bdch,8)

path='/home/noah/Desktop/large_analysis/ca_analysis/cur_data/'
runs=[run for run in os.listdir(path) if run.endswith('all.dat')]
runs.sort()

for run in runs:
    fig,ax=subplots(nrows=2,figsize=(30,20),num=1,sharex=True)
    print('\r'+20*' ',end='')
    print('\r'+run,end='')
    data=(fr.gen_output(path+run)[0]).view(np.recarray)
    data=data[data.energy>100]
    data.energy=pd.vec_calibrate(data.energy,data.board,data.channel)
    data=pd.precuts(data,energy=50)

    t=(data.board==bd)*(data.channel==ch)
    h=histogram(data.energy[t],bins=nbins,range=rang)[0].astype(float)
    t=(b0.board==bd)*(b0.channel==ch)
    b=histogram(b0.energy[t],bins=nbins,range=rang)[0].astype(float)
    
    normbin=np.nonzero(bins==71)[0][0]
    norm=np.sum(h[normbin-1:normbin+1])/np.sum(b[normbin-1:normbin+1])



    ax[0].plot(bins,h,ds='steps',label='data')
    ax[0].plot(bins,b*norm,ds='steps',label='sim')
    ax[0].set_xlim(50,255)
    ax[0].set_ylim(0,3e3)
    ax1=ax[0].twinx()
    ax1.plot(bins,h-b*norm,ds='steps',c='r')
    ax1.set_ylim(-250,500)
    ax1.tick_params('y',labelsize=fs,colors='r')
    ax[0].tick_params(labelsize=fs)
    ax[0].set_title(run[:-8],fontsize=fs)
    ax[0].legend(fontsize=fs)
    
    normbin=np.nonzero(bins==191)[0][0]
    norm=np.sum(h[normbin-1:normbin+1])/np.sum(b[normbin-1:normbin+1])    
    
    ax[1].plot(bins,h,ds='steps',label='data')
    ax[1].plot(bins,b*norm,ds='steps',label='sim')
    ax[1].set_xlim(50,255)
    ax[1].set_ylim(0,3e3)
    ax1=ax[1].twinx()
    ax1.plot(bins,h-b*norm,ds='steps',c='r')
    ax1.set_ylim(-250,500)
    ax1.tick_params('y',labelsize=fs,colors='r')
    ax[1].tick_params(labelsize=fs)
    ax[1].set_title(run[:-8],fontsize=fs)
    ax[1].set_xlabel('Energy [keV]',fontsize=fs)
    ax[1].legend(fontsize=fs)
    tight_layout()
    savefig('./all_the_figs/'+run[:-8]+'-norm_before_after')
    close(fig=1)
# show()

Run_99-all.dat      